Goal: Get simulation to work in most straightforward way.

In [1]:
import numpy as np

side = 10
scatter_radius = 4
particle_radius = 2
num_walls = 5
hole = 3
max_steps = 1000

abs_tol = 1e-5
rel_tol = 0.01

x_range = side - particle_radius
y_range = side - particle_radius
scatter_range = scatter_radius + particle_radius
hole_range = hole - particle_radius

normals = [np.array([1,0])
           ,np.array([-1,0])
           ,np.array([0,1])
           ,np.array([0,-1])
          ]

x = 0.0
y = -y_range
speed = 1.0
theta = np.pi/4
pos = np.array([x,y])
vel = speed * np.array([np.cos(theta), np.sin(theta)])

t = 0.0
escapes = 0
t_hist = [t]
pos_hist = [pos.copy()]
vel_hist = [vel.copy()]

dts = np.zeros(num_walls)
prior_collision = num_walls
for step in range(max_steps):
    dts[:] = np.inf

    if abs(vel[0]) > abs_tol:
        dts[0] = (-x_range - pos[0]) / vel[0]
        dts[1] = ( x_range - pos[0]) / vel[0]

    if abs(vel[1]) > abs_tol:
        dts[2] = (-y_range - pos[1]) / vel[1]
        dts[3] = ( y_range - pos[1]) / vel[1]
    if prior_collision in range(0,4):
        dts[prior_collision] = np.inf

    a = vel.dot(vel)
    if a  > abs_tol:
        b = 2 * pos.dot(vel)
        c = pos.dot(pos) - scatter_range**2
        
        d = b**2 - 4*a*c
        if d < abs_tol:
            small, big = np.inf, np.inf
        else:
            e = np.sqrt(d)
            small = (-b - e) / (2*a)
            big   = (-b + e) / (2*a)
            if b > 0:
                small, big = big, small

        if prior_collision == 4:
            dts[4] = big
        else:
            dts[4] = small

    dts[dts<abs_tol] = np.inf
    col_wall = np.argmin(dts)
    dt = dts[col_wall]
    t = t + dt
    pos = pos + vel*dt

    if col_wall in range(0,4):
        n = normals[col_wall]
    elif col_wall == 4:
        n = pos.copy()
        l = np.sqrt(n.dot(n))
        n = n / l

    vel = vel - 2 * (vel.dot(n)) * n
    prior_collision = col_wall
    
    if col_wall == 2:
        if abs(pos[0]) < hole_range:
            escapes += 1

    t_hist.append(t)
    pos_hist.append(pos.copy())
    vel_hist.append(vel.copy())

    if escapes >= 1:
        break

num_col = len(t_hist)-1
# t_hist = np.array(t_hist)
# pos_hist = np.array(pos_hist)
# vel_hist = np.array(vel_hist)

In [2]:
import matplotlib.pyplot as plt
import ipywidgets as widgets

h = hole_range
x = x_range
y = y_range
side_bdy = np.array([(h,-y), (x,-y), (x,y), (-x,y), (-x,-y), (-h,-y)])
thetas = np.linspace(0, 2*np.pi, 100)
scatter_bdy = scatter_range * np.array([np.cos(thetas), np.sin(thetas)]).T

pos = np.array(pos_hist)
t = t_hist
dpos = np.diff(pos,axis=0)
max_steps = dpos.shape[0]
def draw(steps=1):
    print('steps = {}, time = {:.2f}'.format(steps, t[steps]))
    fig, ax = plt.subplots(figsize=[5,5])
    ax.plot(side_bdy[:,0], side_bdy[:,1])
    ax.fill(scatter_bdy[:,0], scatter_bdy[:,1])
    ax.quiver(pos[:steps,0], pos[:steps,1], dpos[:steps,0], dpos[:steps,1], angles='xy', scale_units='xy', scale=1)
    ax.set_aspect('equal')
    plt.show()
    
widgets.interact(draw,steps=(1,max_steps));

A Jupyter Widget